# Import

In [1]:
from decouple import Config, RepositoryEnv
from ollama_interaction import generate_ollama_request

import pandas as pd
import json

## Import des données sur les modèles
Nous importons les modèles servant de base pour VApp. Par défaut, les modèles sont quantifiés en int4.

In [2]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


In [3]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

## Import data base de travail
Import de la base de données générée dans generation-question.ipynb.

In [4]:
data_question = pd.read_csv("hard-database/data_question.csv",index_col=0)

### Selection des données de test
Pour le moment, le problème est simplifié en se concentrant sur un modèle à trois questions avec trois réponses simples. Les LLM étant connus pour générer de très bons résumés, nous n'avons pas jugé nécessaire d'aller beaucoup plus loin.

In [5]:
project_id = 0


project_description = data_question['project_description'][project_id]
question_list = data_question[['q1','q2','q3']].iloc[project_id].values

print(project_description)
print(question_list)

Revitalisation d'une zone humide
["Votre projet semble se concentrer sur la revitalisation d’une zone humide. Comment ce travail s’inscrit-il dans l'objectif de développement des filières émergentes liées à l'énergie et/ou vers une autonomie alimentaire, comme mentionné par le programme LEADER?"
 "Quels types spécifiques d’équipements ou travaux envisagez-vous pour la revitalisation numérique au sein du service communal qui seraient subventionnables dans le cadre de l'aide Provence Numérique ? Pouvez vous les détailler ?"
 "Afin que votre projet puisse être considéré par LEADER, pouvez-vous expliquer comment sa réalisation contribuera-t-elle à la relocalisation d'activités ou au développement des circuits courts de proximité sur le territoire?"]


In [6]:
dumpy_answer_list = [
    """La revitalisation de la zone humide inclura des aménagements pour améliorer les circulations piétonnes, notamment à travers des sentiers et des passerelles permettant une meilleure accessibilité pour les visiteurs. Ces aménagements favoriseront également une meilleure gestion de la circulation routière aux abords de la zone, réduisant les risques pour les piétons et encourageant un accès sécurisé aux espaces naturels""",
    """Le projet est situé dans une commune comptant moins de 10 000 habitants, ce qui permet de bénéficier d’une aide prioritaire dans le cadre de cette subvention. La subvention d’investissement pourrait ainsi couvrir une part significative des coûts de revitalisation, renforçant les actions locales pour la préservation des écosystèmes et l’attractivité de la commune""",
    """La revitalisation de la zone humide s'inscrit dans les objectifs de la stratégie touristique départementale en valorisant les espaces naturels comme attraits pour les visiteurs. Ce projet vise à développer un écotourisme durable, en sensibilisant les visiteurs à la préservation des milieux naturels tout en renforçant l'attractivité locale grâce à des équipements respectueux de l'environnement et des parcours éducatifs pour les familles et les éco-touristes""",
    ]

## Usage de la fonction gen_prompt_project_summary

In [7]:
# Fonction disponible sous ./prompt_script/gen_prompt_project_summary.py
from prompt_script.gen_prompt_project_summary import gen_prompt_project_summary

### Exemple d'usage
Pour générer des résumés plus pertinents, nous avons besoin d'un LLM plus "intelligent". Notre choix s'est orienté vers Qwen 2.5-14B, qui représente un bon compromis entre vitesse, performance et intelligence.

In [8]:
model = "qwen2.5:14b"
model_options = model_data[model]


request_options = {
    "num_ctx": 16384,
    "num_predict": 2048
}

prompt_system, prompt_user = gen_prompt_project_summary(project_description,question_list,answer_list=dumpy_answer_list)

response = generate_ollama_request(
    prompt_system=prompt_system,
    response_format=None,
    prompt_user=prompt_user,
    ollama_api_url=ollama_api_url,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    model=model, 
    request_options= request_options,  # Default to None
    seed=0,
    )

print(response['response'])

Le projet vise à revitaliser une zone humide située dans un petit village, en améliorant son accessibilité tout en préservant la biodiversité. Les efforts se concentreront sur l'installation de sentiers piétonniers et des passerelles pour sécuriser les déplacements autour du site naturel.

L'aménagement comprendra également une meilleure gestion routière, réduisant le risque d'accidents entre véhicules automobiles et usagers de la route à pied. Ces améliorations visent non seulement l’attrait touristique mais aussi des aspects pratiques pour les résidents locaux qui fréquentent régulièrement cet espace vert.

La revitalisation contribuera directement au développement d'un écotourisme durable et respectueux de la nature. En plus, elle permettra le renforcement du sens civique envers l'environnement par des parcours pédagogiques pour les familles ainsi que diverses activités adaptées aux visiteurs curieux.

Les actions prévues sont conçus dans une optique d'attractivité durable de la com